Example that shows how the `use_orthogonal_projector` option in `skcosmo.linear_model.OrthogonalRegression` can result in nonanalytic behavior of the prediction with respect to changes in the feature dimension.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skcosmo.linear_model import OrthogonalRegression

cube = np.array(
    [
        [0, 0, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 1, 0],
        [0, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
    ]
)

# Can be seen as a cube with zero z-scaling without the z-dimension.
# This is important because we compare it with a cube with zero
# z-scaling keeping the z-dimension
xy_plane_projected_cube = cube[:,[0,1]]


def z_scaled_cube(z_scaling):
    return np.array(
        [
            [0, 0, 0],
            [1, 0, 0],
            [0, 1, 0],
            [0, 0, z_scaling],
            [1, 1, 0],
            [0, 1, z_scaling],
            [1, 0, z_scaling],
            [1, 1, z_scaling],
        ]
    )


z_scalings = np.linspace(0, 1, 10)

We compute the orthogonal regression error for the orthogonal regression of a cube fitting on a cube with z-scaling in the range [0,1]. For the case of a zero z-scaling the error is computed one time with a third dimension and one time without. The regression is done with `skcosmo.linear_model.OrthogonalRegression` first setting the `use_orthogonal_projector` to True.

In [ ]:
regression_errors_for_z_scaled_cube_using_orthogonal_projector = []
orth_reg_using_orthogonal_projector = OrthogonalRegression(use_orthogonal_projector=True)
for z in z_scalings:
    orth_reg_using_orthogonal_projector.fit(cube, z_scaled_cube(z))
    orth_reg_pred_cube = orth_reg_using_orthogonal_projector.predict(cube)
    regression_error = np.linalg.norm(z_scaled_cube(z) - orth_reg_pred_cube)
    regression_errors_for_z_scaled_cube_using_orthogonal_projector.append(regression_error)


orth_reg_using_orthogonal_projector.fit(cube, xy_plane_projected_cube)
orth_reg_pred_cube = orth_reg_using_orthogonal_projector.predict(cube)
regression_error_for_xy_plane_projected_cube_using_orthogonal_projector = (
        np.linalg.norm(xy_plane_projected_cube - orth_reg_pred_cube)
    )

Then setting `use_orthogonal_projector` to False.

In [ ]:
orth_reg = OrthogonalRegression(use_orthogonal_projector=False)
regression_errors_for_z_scaled_cube_zero_padded = []
for z in z_scalings:
    orth_reg.fit(cube, z_scaled_cube(z))
    orth_reg_pred_cube = orth_reg.predict(cube)
    regression_error = np.linalg.norm(z_scaled_cube(z) - orth_reg_pred_cube)
    regression_errors_for_z_scaled_cube_zero_padded.append(regression_error)

Setting the `use_orthogonal_projector` option to False pads automatically input and output data to the same dimension with zeros. Therefore we pad `xy_plane_projected_cube` to three dimensions with zeros.

In [ ]:
orth_reg.fit(cube, xy_plane_projected_cube)
orth_reg_pred_cube = orth_reg.predict(cube)
zero_padded_xy_plane_projected_cube = np.pad(xy_plane_projected_cube, [(0, 0), (0, 1)])

regression_error_for_xy_plane_projected_cube_zero_padded = np.linalg.norm(
    zero_padded_xy_plane_projected_cube - orth_reg_pred_cube
)

It can be seen that setting the  `use_orthogonal_projector` to True results in a nonanalytic behavior with respect to changes in the dimension. The zero z-scaling case returns different orthogonal regression errors depeding on retaining the third dimension or not. When setting the `use_orthogonal_projector` to False this nonanalytic behavior is not present.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 3.8), sharey=True)

axes[0].plot(
    z_scalings,
    regression_errors_for_z_scaled_cube_using_orthogonal_projector,
    label="Regression error for z-scaled cube",
)
axes[0].scatter(
    0,
    regression_error_for_xy_plane_projected_cube_using_orthogonal_projector,
    label="Regression error for xy_plane_projected_cube",
)
axes[0].set_title(
    "Orthogonal regression erro for\n features using orthogonal projector"
)
axes[0].set_xlabel("scaling in z direction")
axes[0].set_ylabel("orthogonal regression error")

axes[1].plot(
    z_scalings,
    regression_errors_for_z_scaled_cube_zero_padded,
    label="Regression error for z-scaled cube",
)
axes[1].scatter(
    0,
    regression_error_for_xy_plane_projected_cube_zero_padded,
    label="Regression error for xy_plane_projected_cube",
)
axes[1].set_title("Orthogonal regression error for\n zero padded features")
axes[1].set_xlabel("scaling in z direction")
axes[1].legend(loc="lower right")
plt.show()